In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # Import for plotting

# --- Configuration ---
# Stock tickers to include
TICKERS = ['BBCA.JK', 'BBRI.JK', 'BMRI.JK', 'TLKM.JK', 'ASII.JK']

RISK_FREE_RATE = 0.02                                # Annual risk-free rate (e.g., 2% for T-bills)
NUM_PORTFOLIOS = 50000                               # Number of portfolios for Monte Carlo Simulation

# --- Helper Functions for Ratio Calculation ---

def calculate_sortino_ratio(returns, weights, risk_free_rate, annualization_factor):
    """Calculates the annualized Sortino Ratio."""
    portfolio_returns = (returns * weights).sum(axis=1)
    # Target return is often the risk-free rate, or zero. We'll use zero for simplicity
    # as the 'Minimum Acceptable Return' (MAR) is often 0 or RFR.
    MAR = 0.0

    # Identify negative deviations (downside risk)
    downside_returns = portfolio_returns[portfolio_returns < MAR]

    if len(downside_returns) == 0:
        # Avoid division by zero if there are no negative returns
        return np.nan

    # Calculate Downside Deviation (Annualized)
    downside_deviation = np.sqrt(np.mean(downside_returns**2)) * np.sqrt(annualization_factor)

    # Calculate Portfolio Annual Return
    annual_return = portfolio_returns.mean() * annualization_factor

    # Sortino Ratio
    sortino_ratio = (annual_return - risk_free_rate) / downside_deviation
    return sortino_ratio

def calculate_calmar_ratio(returns, weights, annualization_factor):
    """Calculates the Calmar Ratio."""
    portfolio_returns = (returns * weights).sum(axis=1)

    # Calculate Cumulative Returns
    cumulative_returns = (1 + portfolio_returns).cumprod()

    # Calculate Max Drawdown
    peak = cumulative_returns.expanding(min_periods=1).max()
    drawdown = (cumulative_returns / peak) - 1.0
    max_drawdown = drawdown.min() * -1 # Make it positive

    # Calculate Portfolio Annual Return
    annual_return = portfolio_returns.mean() * annualization_factor

    if max_drawdown == 0:
        return np.nan

    # Calmar Ratio
    calmar_ratio = annual_return / max_drawdown
    return calmar_ratio


# --- Main Logic ---

def portfolio_optimizer(tickers, risk_free_rate, num_portfolios):
    """
    Downloads data, performs Monte Carlo simulation, and finds optimal portfolios.
    Returns the results DataFrame and a dictionary of optimal portfolios.
    """

    try:
        data = yf.download(tickers, start='2015-01-01', end='2025-01-01', progress=False, auto_adjust=True)['Close']
    except Exception as e:
        print(f"Error downloading data: {e}")
        return None, None

    if data.empty:
        print("Data is empty. Check tickers or date range.")
        return None, None

    # Daily returns (log returns are better for portfolio math, but simple returns often used for demonstration)
    returns = data.pct_change().dropna()

    # Determine the annualization factor (252 trading days for stocks)
    annualization_factor = 252

    # Pre-calculate annualized portfolio return and covariance matrix
    # Annualized daily returns: mean return * 252
    mean_daily_returns = returns.mean()
    # Annualized covariance: covariance * 252
    cov_matrix = returns.cov() * annualization_factor

    # MONTE CARLO SIMULATION
    # Arrays to store results
    portfolio_results = np.zeros((num_portfolios, len(tickers) + 5)) # +5 for Return, Volatility, Sharpe, Sortino, Calmar

    for i in range(num_portfolios):
        # Generate random weights
        weights = np.random.random(len(tickers))
        weights /= np.sum(weights)

        # Calculate Annualized Portfolio Statistics

        # Annual Return: dot product of (weights * annualized mean returns)
        portfolio_return = np.sum(mean_daily_returns * weights) * annualization_factor

        # Annual Volatility: sqrt((weights_transpose * cov_matrix * weights))
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

        # Sharpe Ratio: (Return - Risk-Free Rate) / Volatility
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility

        # Sortino Ratio (using the custom function)
        sortino_ratio = calculate_sortino_ratio(returns, weights, risk_free_rate, annualization_factor)

        # Calmar Ratio (using the custom function)
        calmar_ratio = calculate_calmar_ratio(returns, weights, annualization_factor)

        # Store the results
        portfolio_results[i, 0] = portfolio_return
        portfolio_results[i, 1] = portfolio_volatility
        portfolio_results[i, 2] = sharpe_ratio
        portfolio_results[i, 3] = sortino_ratio
        portfolio_results[i, 4] = calmar_ratio
        # Store weights (starting from index 5)
        for j in range(len(weights)):
            portfolio_results[i, j + 5] = weights[j]

    # Convert results to DataFrame for easy analysis
    column_names = ['Return', 'Volatility', 'Sharpe_Ratio', 'Sortino_Ratio', 'Calmar_Ratio'] + [f'Weight_{t}' for t in tickers]
    results_df = pd.DataFrame(portfolio_results, columns=column_names)

    # FIND OPTIMAL PORTFOLIOS
    # Max Sharpe Ratio Portfolio
    max_sharpe_portfolio = results_df.iloc[results_df['Sharpe_Ratio'].idxmax()]

    # Max Sortino Ratio Portfolio
    max_sortino_portfolio = results_df.iloc[results_df['Sortino_Ratio'].idxmax()]

    # Max Calmar Ratio Portfolio
    max_calmar_portfolio = results_df.iloc[results_df['Calmar_Ratio'].idxmax()]

    # Min Volatility Portfolio
    min_volatility_portfolio = results_df.iloc[results_df['Volatility'].idxmin()]

    # Dictionary to hold optimal portfolios for easy access in plotting
    optimal_portfolios = {
        'Max Sharpe': max_sharpe_portfolio,
        'Min Volatility': min_volatility_portfolio,
        'Max Sortino': max_sortino_portfolio,
        'Max Calmar': max_calmar_portfolio
    }

    # DISPLAY RESULTS
    def display_portfolio(title, portfolio):
        print(f"\n--- {title} ---")
        print(f"Annual Return:        {portfolio['Return']:.2%}")
        print(f"Annual Volatility:    {portfolio['Volatility']:.2%}")
        print(f"Sharpe Ratio:         {portfolio['Sharpe_Ratio']:.4f}")
        print(f"Sortino Ratio:        {portfolio['Sortino_Ratio']:.4f}")
        print(f"Calmar Ratio:         {portfolio['Calmar_Ratio']:.4f}")
        print("\nAllocation Weights:")
        weights_info = {ticker: portfolio[f'Weight_{ticker}'] for ticker in tickers}
        # Sort weights for better readability
        sorted_weights = sorted(weights_info.items(), key=lambda item: item[1], reverse=True)
        for ticker, weight in sorted_weights:
            print(f"  {ticker:<5}: {weight:.2%}")

    print(f"--- Portfolio Optimization Results ({num_portfolios} Simulations) ---")
    print(f"Tickers: {', '.join(tickers)}")
    #print(f"Data Period: Last {n_years} Years")
    print(f"Risk-Free Rate: {risk_free_rate:.2%}")

    # Display the results for the optimal portfolios
    display_portfolio("Portfolio with MAXIMUM SHARPE RATIO", max_sharpe_portfolio)
    display_portfolio("Portfolio with MINIMUM VOLATILITY", min_volatility_portfolio)
    display_portfolio("Portfolio with MAXIMUM SORTINO RATIO", max_sortino_portfolio)
    display_portfolio("Portfolio with MAXIMUM CALMAR RATIO", max_calmar_portfolio)

    # Return the full DataFrame and optimal portfolios for plotting
    return results_df, optimal_portfolios

# --- Execute the Optimizer ---
if __name__ == '__main__':
    # You might need to install the libraries first:
    # pip install yfinance pandas numpy scipy matplotlib

    results, optimal_ports = portfolio_optimizer(
        tickers=TICKERS,
        risk_free_rate=RISK_FREE_RATE,
        num_portfolios=NUM_PORTFOLIOS
    )

    # Plot the results (requires matplotlib)
    if results is not None:
        # 1. Create the initial scatter plot
        plt.figure(figsize=(12, 7))
        scatter = plt.scatter(
            results['Volatility'],
            results['Return'],
            c=results['Sharpe_Ratio'], # Color points by Sharpe Ratio
            cmap='viridis',
            label='Simulated Portfolios',
            s=10 # Smaller size for many points
        )
        plt.colorbar(scatter, label='Sharpe Ratio')

        # 2. Mark the optimal portfolios
        # Define optimal portfolio data and markers
        ports_to_plot = [
            {'label': 'Max Sharpe Ratio', 'data': optimal_ports['Max Sharpe'], 'color': 'red', 'marker': '*'},
            {'label': 'Min Volatility', 'data': optimal_ports['Min Volatility'], 'color': 'blue', 'marker': 'D'},
            {'label': 'Max Sortino Ratio', 'data': optimal_ports['Max Sortino'], 'color': 'magenta', 'marker': '^'},
            {'label': 'Max Calmar Ratio', 'data': optimal_ports['Max Calmar'], 'color': 'cyan', 'marker': 'v'},
        ]

        # Plot each optimal portfolio
        for p in ports_to_plot:
            plt.scatter(
                p['data']['Volatility'],
                p['data']['Return'],
                color=p['color'],
                marker=p['marker'],
                s=100, # Larger size for emphasis
                label=p['label']
            )

        # 3. Add titles, labels, and legend
        plt.title('Efficient Frontier - Monte Carlo Simulation', fontsize=14)
        plt.xlabel('Annualized Volatility (Standard Deviation)', fontsize=12)
        plt.ylabel('Annualized Expected Return', fontsize=12)
        plt.legend(loc='best')
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.show()